In [ ]:
import glob, os, re
import matplotlib.pyplot as plt

min_latencies = []
max_len = 8
#tests = ['20190723_OpenTuner_1', '20190723_Gaussian_process_1', '20190722_combined_model']
#tests = ['20190723_OpenTuner_1', '20190723_OpenTuner_2', '20190723_OpenTuner_3', '20190723_Gaussian_process_1',
#         '20190723_Gaussian_process_2', '20190723_Gaussian_process_3', '20190722_combined_model']
tests = ['20190723_OpenTuner_', '20190723_Gaussian_process_', '20190722_combined_model']
for test in tests:
    test_dirs = glob.glob('/proj/xsjhdstaff5/hansg/Tests/Unroll_toy/' + test + '*')
    all_latencies = []
    for test_dir in test_dirs:
        src_dirs = glob.glob(test_dir + '/Output/Sources/*')
        src_dirs.sort()

        latency_pattern = re.compile(r'\$AVER_LATENCY = "(\d+)"')

        factors = []
        latencies = []
        for src_dir in src_dirs:
            output_dir = test_dir + '/Output/Output/' + os.path.basename(src_dir)
            reports = glob.glob(output_dir + '/*/proj/sol1/sim/report/verilog/lat.rpt')
            if len(reports) != 1:
                continue
            with open(reports[0], 'r') as input_file:
                text = input_file.read()
            match = re.search(latency_pattern, text)
            latency = int(match.group(1))
            latencies.append(latency)
            if len(latencies) == max_len:
                break

        all_latencies.append(np.minimum.accumulate(latencies))

    min_latencies.append(np.mean(np.array(all_latencies), axis = 0))

for latencies in min_latencies:
    plt.plot(range(1, max_len + 1), latencies[range(max_len)])
plt.title('Latency of loop unrolling')
plt.xlabel('Iteration')
plt.ylabel('Latency')
plt.yscale('log')
plt.xlim(0, max_len + 1)
plt.legend(['OpenTuner default', 'Bayesian optimization', 'Proposed algorithm'])
plt.show()

best_latency = min([min(latencies) for latencies in min_latencies])
for latencies in min_latencies:
    deltas = [latency - best_latency for latency in latencies]
    deltas = [delta if delta > 0 else 1e-7 for delta in deltas]
    plt.plot(range(1, max_len + 1), deltas[0 : max_len])
plt.title('Latency difference w.r.t. optimum of loop unrolling')
plt.xlabel('Iteration')
plt.ylabel('Latency difference')
plt.yscale('log')
plt.xlim(0, max_len + 1)
plt.legend(['OpenTuner default', 'Bayesian optimization', 'Proposed algorithm'])
plt.show()